In [11]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import time
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

#VGG 16 and Resnet use 3 channel rgb inputs
transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Lambda(lambda x: x.repeat(3, 1, 1)),  # Convert 1 channel to 3 channels
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])  # Normalize for RGB

trainset = torchvision.datasets.FashionMNIST(root='./data', train=True,
                                             download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                           shuffle=True)

testset = torchvision.datasets.FashionMNIST(root='./data', train=False,
                                            download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                          shuffle=False)

cuda


In [13]:
vgg16Fashion = models.vgg16(pretrained=True)
vgg16Fashion.classifier[6] = nn.Linear(4096, 10)
vgg16Fashion.to(device)
print(vgg16Fashion)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [14]:
# optimizer
optimizer = optim.SGD(vgg16Fashion.classifier.parameters(), lr=0.001, momentum=0.9)
# loss function
criterion = nn.CrossEntropyLoss()

In [15]:
#transformations for metamorphic testing
transformations = [
    transforms.RandomRotation(degrees=10),  # Small rotation
    transforms.ColorJitter(brightness=0.2),  # Slight brightness change
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1))  # Small translation
]

In [16]:
# Validation Function
def validate(model, test_dataloader):
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for data, target in tqdm(test_dataloader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            val_running_loss += loss.item()
            _, preds = torch.max(output.data, 1)
            val_running_correct += (preds == target).sum().item()
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
    
    val_loss = val_running_loss / len(test_dataloader.dataset)
    val_accuracy = 100. * val_running_correct / len(test_dataloader.dataset)
    
    # Calculate metrics
    precision = precision_score(all_targets, all_preds, average='weighted')
    recall = recall_score(all_targets, all_preds, average='weighted')
    f1 = f1_score(all_targets, all_preds, average='weighted')
    
    return val_loss, val_accuracy, precision, recall, f1

In [17]:
def fit(model, train_dataloader):
    model.train()
    train_running_loss = 0.0
    train_running_correct = 0
    for  data, target in tqdm(train_dataloader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        train_running_loss += loss.item()
        _, preds = torch.max(output.data, 1)
        train_running_correct += (preds == target).sum().item()
        loss.backward()
        optimizer.step()
    train_loss = train_running_loss/len(train_dataloader.dataset)
    train_accuracy = 100. * train_running_correct/len(train_dataloader.dataset)
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}')
    
    return train_loss, train_accuracy

In [18]:
reject_diffs = []

# If batch_size > 1, we get multiple images each iteration.
for i, (image, label) in enumerate(tqdm(testloader)):
   
    image = image.to(device)


    with torch.no_grad():
        original_output = vgg16Fashion(image)
        original_prob = F.softmax(original_output, dim=1)
        original_confidence, original_pred = torch.max(original_prob, dim=1)  # shape [batch_size]

    
    for tf in transformations:
        transformed_image = tf(image.cpu()).to(device)
        
        with torch.no_grad():
            transformed_output = vgg16Fashion(transformed_image)
            transformed_prob = F.softmax(transformed_output, dim=1)
            transformed_confidence, transformed_pred = torch.max(transformed_prob, dim=1)
        
        # Compute absolute difference in confidence for the entire batch
        # shape [batch_size]
        diff_tensor = torch.abs(original_confidence - transformed_confidence)

        
        diff_mean = diff_tensor.mean().item()
        reject_diffs.append(diff_mean)

        
        print(f"Batch {i} | "
              f"Reject Differential (mean over batch): {diff_mean:.4f}")

  0%|                                                                                              | 0/313 [00:00<?, ?it/s]

Batch 0 | Reject Differential (mean over batch): 0.0491
Batch 0 | Reject Differential (mean over batch): 0.0374


  0%|▎                                                                                     | 1/313 [00:00<02:42,  1.91it/s]

Batch 0 | Reject Differential (mean over batch): 0.0282
Batch 1 | Reject Differential (mean over batch): 0.0332
Batch 1 | Reject Differential (mean over batch): 0.0453


  1%|▌                                                                                     | 2/313 [00:00<02:29,  2.08it/s]

...

Batch 310 | Reject Differential (mean over batch): 0.0450
Batch 310 | Reject Differential (mean over batch): 0.0465
Batch 310 | Reject Differential (mean over batch): 0.0409
Batch 311 | Reject Differential (mean over batch): 0.0406
Batch 311 | Reject Differential (mean over batch): 0.0405


100%|███████████████████████████████████████████████████████████████████████████████████▋| 312/313 [02:10<00:00,  2.41it/s]

Batch 311 | Reject Differential (mean over batch): 0.0380
Batch 312 | Reject Differential (mean over batch): 0.0345
Batch 312 | Reject Differential (mean over batch): 0.0527


100%|████████████████████████████████████████████████████████████████████████████████████| 313/313 [02:10<00:00,  2.40it/s]

Batch 312 | Reject Differential (mean over batch): 0.0425


In [37]:
model_save_path = './saved_models/vgg16Fashion_Fashion-MNIST_finetuned.pth'
# Save the model
torch.save(vgg16Fashion.state_dict(), model_save_path)
print("Model saved as vgg16Fashion_Fashion-MNIST_finetuned.pth")

Model saved as vgg16Fashion_Fashion-MNIST_finetuned.pth


In [38]:
vgg16Fashion.load_state_dict(torch.load(model_save_path, weights_only=True))
vgg16Fashion = vgg16Fashion.to(device)

In [39]:
# Test the final metrics on the test set
final_val_loss, final_val_accuracy, final_precision, final_recall, final_f1 = validate(vgg16Fashion, testloader)
print(f'\nFinal Test Results - Loss: {final_val_loss:.4f}, Accuracy: {final_val_accuracy:.2f}%, Precision: {final_precision:.2f}, Recall: {final_recall:.2f}, F1 Score: {final_f1:.2f}')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:36<00:00,  8.53it/s]



Final Test Results - Loss: 0.0065, Accuracy: 92.76%, Precision: 0.93, Recall: 0.93, F1 Score: 0.93
